# Wrangle File for Clustering with the Zillow Data

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Statistical Tests
import scipy.stats as stats

# Visualizing
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import learning_curve

import wrangle_excs as wrangle

In [2]:
sql = '''
SELECT * 
FROM properties_2017 AS prop
JOIN (SELECT parcelid, MAX(transactiondate) as transactiondate
    FROM predictions_2017
    GROUP BY parcelid) AS txn ON prop.parcelid = txn.parcelid      
JOIN predictions_2017 AS pred ON prop.parcelid = pred.parcelid 
    AND pred.transactiondate = txn.transactiondate 
LEFT JOIN airconditioningtype AS ac ON prop.airconditioningtypeid = ac.airconditioningtypeid
LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
LEFT JOIN buildingclasstype build USING (buildingclasstypeid)
LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid)
LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid)
LEFT JOIN storytype story USING (storytypeid)
LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid)
WHERE  prop.latitude IS NOT NULL
    AND prop.longitude IS NOT NULL;
'''

In [3]:
sql_alt = """
SELECT prop.*,
       pred.logerror,
       pred.transactiondate,
       air.airconditioningdesc,
       arch.architecturalstyledesc,
       build.buildingclassdesc,
       heat.heatingorsystemdesc,
       landuse.propertylandusedesc,
       story.storydesc,
       construct.typeconstructiondesc
FROM   properties_2017 prop
       INNER JOIN (SELECT parcelid,
                   Max(transactiondate) transactiondate
                   FROM   predictions_2017
                   GROUP  BY parcelid) pred
               USING (parcelid)
            JOIN predictions_2017 as pred USING (parcelid, transactiondate)
       LEFT JOIN airconditioningtype air USING (airconditioningtypeid)
       LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
       LEFT JOIN buildingclasstype build USING (buildingclasstypeid)
       LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid)
       LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid)
       LEFT JOIN storytype story USING (storytypeid)
       LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid)
WHERE  prop.latitude IS NOT NULL
       AND prop.longitude IS NOT NULL;
"""

In [4]:
def get_db_url(database):
    from env import hostname, username, password
    url = f'mysql+pymysql://{username}:{password}@{hostname}/{database}'
    return url

In [7]:
# acquire zillow data using the query
def get_zillow(sql_alt):
    url = get_db_url('zillow')
    zillow_df = pd.read_sql(sql, url)
    return zillow_df

In [8]:
zillow = get_zillow(sql)

In [9]:
zillow.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,id,parcelid,airconditioningtypeid,basementsqft,...,logerror,transactiondate,airconditioningtypeid,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,NaN,NaN,NaN,1727539,14297519,NaN,NaN,...,0.025595,2017-01-01,NaN,None,None,None,None,Single Family Residential,None,None
1,NaN,NaN,261.0,NaN,NaN,NaN,1387261,17052889,NaN,NaN,...,0.055619,2017-01-01,NaN,None,None,None,None,Single Family Residential,None,None
2,NaN,NaN,261.0,NaN,NaN,NaN,11677,14186244,NaN,NaN,...,0.005383,2017-01-01,NaN,None,None,None,None,Single Family Residential,None,None
3,NaN,NaN,261.0,2.0,NaN,NaN,2288172,12177905,NaN,NaN,...,-0.103410,2017-01-01,NaN,None,None,None,Central,Single Family Residential,None,None
4,NaN,NaN,266.0,2.0,NaN,NaN,1970746,10887214,1.0,NaN,...,0.006940,2017-01-01,1.0,Central,None,None,Central,Condominium,None,None


In [10]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 73 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        222 non-null    float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77381 non-null  float64
 3   heatingorsystemtypeid         49440 non-null  float64
 4   buildingclasstypeid           15 non-null     float64
 5   architecturalstyletypeid      206 non-null    float64
 6   id                            77381 non-null  int64  
 7   parcelid                      77381 non-null  int64  
 8   airconditioningtypeid         24953 non-null  float64
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77381 non-null  float64
 11  bedroomcnt                    77381 non-null  float64
 12  buildingqualitytypeid         49672 non-null  float64
 13  c

In [11]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .70):
	#function that will drop rows or columns based on the percent of values that are missing:\
	#handle_missing_values(df, prop_required_column, prop_required_row
    threshold = int(round(prop_required_column*len(df.index),0))
    df = df.dropna(axis=1, thresh=threshold)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [13]:
zillow = handle_missing_values(zillow)
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77367 entries, 0 to 77380
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   propertylandusetypeid         77367 non-null  float64
 1   heatingorsystemtypeid         49440 non-null  float64
 2   id                            77367 non-null  int64  
 3   parcelid                      77367 non-null  int64  
 4   bathroomcnt                   77367 non-null  float64
 5   bedroomcnt                    77367 non-null  float64
 6   buildingqualitytypeid         49672 non-null  float64
 7   calculatedbathnbr             76772 non-null  float64
 8   calculatedfinishedsquarefeet  77185 non-null  float64
 9   finishedsquarefeet12          73749 non-null  float64
 10  fips                          77367 non-null  float64
 11  fullbathcnt                   76772 non-null  float64
 12  latitude                      77367 non-null  float64
 13  l

In [12]:
def remove_columns(df, cols_to_remove):
    #remove columns not needed
    df = df.drop(columns=cols_to_remove)
    return df